In [3]:
import gensim

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [5]:
import MeCab
import re

mecab = MeCab.Tagger ()
pattern = re.compile('[a-zA-Z]')
noun_list = []

i = 0

for text in model.wv.index2word:
    #print("text", text)
        
    mecab.parse('')#文字列がGCされるのを防ぐ
    node = mecab.parseToNode(text)
    
    b = re.search(pattern,text)  
    #print("result", result)
        
    if b: 
        #print("英語")
        #print(result.group())
        continue 
    
    i +=1
    
    while node:
        #単語を取得
        word = node.surface
        
        #品詞を取得
        pos = node.feature.split(",")[1]
        #print('{0} , {1}'.format(word, pos))
        
        #print("pos", pos)
        #if pos == "一般" or pos == "固有名詞":
        if pos == "一般":
            noun_list.append(word)
        
        #次の単語に進める
        node = node.next
        #print("while")

In [6]:
len(noun_list)

176508

In [7]:
len(set(noun_list))

43413

In [8]:
import pickle

f = open('ippann_noun_list.binaryfile','wb')
noun_list = list(set(noun_list))
pickle.dump(noun_list,f)
f.close

<function BufferedWriter.close>

# 単語ベクトルを辞書で取得

In [6]:
import pickle

f = open('ippann_noun_list.binaryfile','rb')
noun_list = pickle.load(f)

In [7]:
#ランダムで取得

import random
n = 1

num = int(len(noun_list) / n) #半分取得
noun_list = random.sample(noun_list, num)

In [8]:
len(noun_list)

43413

In [9]:
noun_dic = {}

for word in noun_list:
    try:
        noun_dic[word] = model.wv[word]
    except:
        continue

In [10]:
len(noun_dic)

0

In [28]:
import pickle

f = open('ippann_noun_dic.binaryfile','wb')
#f = open('koyuu_noun_sample_dic.binaryfile','wb')
pickle.dump(noun_dic,f)
f.close

<function BufferedWriter.close>

# 単語ベクトル集合を算出

In [11]:
import pickle

f = open('ippann_noun_dic.binaryfile','rb')
noun_dic= pickle.load(f)

In [12]:
import numpy as np

noun_vec_list = list(noun_dic.values())

re_noun_vec_list = []

In [13]:
import random 
n = 1

sample_list = random.sample(noun_vec_list, int(len(noun_vec_list)/n))

In [14]:
len(sample_list)

40853

In [15]:
for i in sample_list:
    a = i.reshape(50,)
    re_noun_vec_list.append(a)

re_noun_vec_list = np.array(re_noun_vec_list)

In [16]:
len(re_noun_vec_list)

40853

In [17]:
list(re_noun_vec_list)

[array([ 0.13991004,  0.11963573, -0.04206443,  0.16350861, -0.15678114,
         0.02663146, -0.40763733,  0.02329282, -0.16533913, -0.05510934,
        -0.06099633, -0.11895447,  0.03718898, -0.2208807 , -0.16946197,
         0.0378987 ,  0.08595221, -0.20326638, -0.11816767,  0.23813471,
        -0.10938767, -0.01557008, -0.271965  ,  0.00574655,  0.18500109,
         0.01143672, -0.00339011,  0.11227683,  0.00621026,  0.1578262 ,
        -0.00952887,  0.0017194 , -0.02640406,  0.01197273,  0.06629634,
         0.2263593 , -0.02751257,  0.06109679,  0.00531614, -0.10715836,
         0.14682405,  0.29813293, -0.00138752, -0.1812686 ,  0.18836859,
         0.01416736,  0.13923723,  0.07708811, -0.23117128,  0.09597288],
       dtype=float32),
 array([ 0.20828645, -0.06626253,  0.13586238,  0.0634352 , -0.17125034,
        -0.06316417, -0.28345233, -0.22058316,  0.1292344 , -0.15692148,
         0.1500549 ,  0.0398501 ,  0.18190712,  0.14558701, -0.06087991,
         0.04437039,  0.166

In [23]:
import pickle

f = open('sample_ippann_noun_vec_list.binaryfile','wb')
#f = open('koyuu_noun_sample_dic.binaryfile','wb')
pickle.dump(list(re_noun_vec_list), f)
f.close

<function BufferedWriter.close>

In [9]:
from sklearn.cluster import DBSCAN
import numpy as np

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(re_noun_vec_list))

for eps in range(9,11):
    eps = eps/10
    
    for minPts in range(1,10):
        
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(re_noun_vec_list)
        
        #print(dbscan)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +2)
        
        # クラスタ1に含まれる点の数
        print("クラスタ1に含まれる点の数", len(np.where(y_dbscan ==0)[0]))
        
        # クラスタ2に含まれる点の数
        print("クラスタ2に含まれる点の数", len(np.where(y_dbscan ==1)[0]))
        
        # クラスタ3に含まれる点の数
        print("クラスタ3に含まれる点の数", len(np.where(y_dbscan ==2)[0]))
        
        print("                            ")
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
    f_name = "./ippann_relation_DBCSAN_fold" + str(eps)
    f=open(f_name, "wb")
    pickle.dump(eps_minPts, f)
    f.close()
    eps_minPts = {}
    minPts_data = {}

noun_vec_list数 :  40853
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 1
outlier数 0
クラスタ数 177
クラスタ1に含まれる点の数 40674
クラスタ2に含まれる点の数 1
クラスタ3に含まれる点の数 1
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 2
outlier数 173
クラスタ数 4
クラスタ1に含まれる点の数 40674
クラスタ2に含まれる点の数 4
クラスタ3に含まれる点の数 2
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 3
outlier数 175
クラスタ数 3
クラスタ1に含まれる点の数 40674
クラスタ2に含まれる点の数 4
クラスタ3に含まれる点の数 0
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 4
outlier数 183
クラスタ数 3
クラスタ1に含まれる点の数 40666
クラスタ2に含まれる点の数 4
クラスタ3に含まれる点の数 0
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 5
outlier数 195
クラスタ数 2
クラスタ1に含まれる点の数 40658
クラスタ2に含まれる点の数 0
クラスタ3に含まれる点の数 0
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 6
outlier数 208
クラスタ数 2
クラスタ1に含まれる点の数 40645
クラスタ2に含まれる点の数 0
クラスタ3に含まれる点の数 0
                            
y_dbscan [0 0 0 ... 0 0 0]
eps: 0.9 ,minPts: 7
outlier数 216
クラスタ数 2
クラスタ1に含まれる点の数 

In [11]:
from sklearn.cluster import DBSCAN
import numpy as np

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(re_noun_vec_list))

for eps in range(7, 8):
    eps = eps/10
    
    for minPts in range(1,10):
        
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(re_noun_vec_list)
        
        #print(dbscan)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +2)
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
    f_name = "./ippann_relation_DBCSAN_fold" + str(eps)
    f=open(f_name, "wb")
    pickle.dump(eps_minPts, f)
    f.close()
    eps_minPts = {}
    minPts_data = {}

noun_vec_list数 :  40853
y_dbscan [   0    0    0 ...    0 7922 7923]
eps: 0.7 ,minPts: 1
outlier数 0
クラスタ数 7925
クラスタ1に含まれる点の数 32451
クラスタ2に含まれる点の数 1
クラスタ3に含まれる点の数 1
                            
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.7 ,minPts: 2
outlier数 7569
クラスタ数 356
クラスタ1に含まれる点の数 32451
クラスタ2に含まれる点の数 2
クラスタ3に含まれる点の数 2
                            
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.7 ,minPts: 3
outlier数 8131
クラスタ数 75
クラスタ1に含まれる点の数 32451
クラスタ2に含まれる点の数 8
クラスタ3に含まれる点の数 3
                            
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.7 ,minPts: 4
outlier数 8609
クラスタ数 32
クラスタ1に含まれる点の数 32114
クラスタ2に含まれる点の数 6
クラスタ3に含まれる点の数 4
                            
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.7 ,minPts: 5
outlier数 9019
クラスタ数 13
クラスタ1に含まれる点の数 31774
クラスタ2に含まれる点の数 4
クラスタ3に含まれる点の数 6
                            
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.7 ,minPts: 6
outlier数 9362
クラスタ数 7
クラスタ1に含まれる点の数 31460
クラスタ2に含まれる点の数 6
クラスタ3に含まれる点の数 10
                            
y_dbscan [ 0  0  0 ...

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(re_noun_vec_list))

for eps in range(6,10):
    
    eps = eps/10 + 0.05
    
    for minPts in range(1,10):
        
        if minPts == 1: continue
        
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(re_noun_vec_list)
        
        #print(dbscan)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +2)
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
    f_name = "./ippann_relation_DBCSAN_fold/" + str(eps)
    f=open(f_name, "wb")
    pickle.dump(eps_minPts, f)
    f.close()
    eps_minPts = {}
    minPts_data = {}



noun_vec_list数 :  40853
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.65 ,minPts: 2
outlier数 12824
クラスタ数 653
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.65 ,minPts: 3
outlier数 13824
クラスタ数 153
y_dbscan [ 0  0  0 ...  0 -1 -1]
eps: 0.65 ,minPts: 4
outlier数 14656
クラスタ数 64
